In [23]:
%load_ext autoreload
%autoreload 2
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import datetime
import pandas as pd
import itertools
import sys
import os
import math
sys.path.append('../..')
import scipy.optimize as opt

from models.optim.gaboh_choice_opt import SEIR

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
# np.array([start days]), np.array([duration]), np.array([choice], number_of_days_simulated)
def calculate_opt(intervention_day, intervention_duration, intervention_choice, days):
    R0 = 2.2 
    T_inf = 2.9
    T_trans = T_inf/R0
    T_inc = 5.2
    T_recov_mild = (14 - T_inf)
    T_hosp = 5
    T_recov_severe = (31.5 - T_inf)
    T_death = 32

    P_severe = 0.2
    P_fatal = 0.02
    P_mild = 1 - P_severe - P_fatal

    N = 7e6
    I0 = 1.0

    assert(len(intervention_day) == len(intervention_duration))
    assert(len(intervention_duration) == len(intervention_choice))
    
    params = [T_trans, T_inc, T_inf, T_recov_mild, T_hosp, T_recov_severe, T_death, 
              P_mild, P_severe, P_fatal, N, intervention_day, intervention_duration, intervention_choice]

    # S, E, I, R_mild, R_severe, R_severe_home, R_fatal, C, D
    state_init_values = [(N - I0)/N, 0, I0/N, 0, 0, 0, 0, 0, 0]
    
    solver = SEIR(params, state_init_values)
    sol = solver.solve_ode(time_step=1, total_no_of_days=days)
    states_int_array = (sol.y*N).astype('int')
    
    E = states_int_array[1]
    I = states_int_array[2]
    H = states_int_array[5]
    F = states_int_array[8]
    
    S_coeficeint=1
    E_coeficeint=0.825
    I_coeficeint=0.75
    R_mild_coeficeint=0.625
    R_severe_coeficeint=0.5
    R_severe_home_coeficeint=0.325
    R_R_fatal_coeficeint=0.25
    C_coeficeint=1
    D_coeficeint=0
    
#     S_coeficeint=1
#     E_coeficeint=0
#     I_coeficeint=0
#     R_mild_coeficeint=0
#     R_severe_coeficeint=0
#     R_severe_home_coeficeint=0
#     R_R_fatal_coeficeint=0
#     C_coeficeint=1
#     D_coeficeint=0
    
    coeficeint=[S_coeficeint,E_coeficeint,I_coeficeint,R_mild_coeficeint,R_severe_coeficeint,R_severe_home_coeficeint,\
                R_severe_home_coeficeint,R_R_fatal_coeficeint,C_coeficeint,D_coeficeint]
    
    objective=0
    for i in range(8):
        objective+=coeficeint[i]*np.sum(states_int_array[i]/1e8)
    return(objective)

In [3]:
fx12 = calculate_opt(intervention_day=np.array([100,151]), intervention_duration=np.array([50,50]), intervention_choice=np.array([0.5,0.5]), days=300)
fx1 = calculate_opt(intervention_day=np.array([100]), intervention_duration=np.array([50]), intervention_choice=np.array([0.5]), days=300)
fx2 = calculate_opt(intervention_day=np.array([151]), intervention_duration=np.array([50]), intervention_choice=np.array([0.5]), days=300)
fx = calculate_opt(intervention_day=np.array([100]), intervention_duration=np.array([50]), intervention_choice=np.array([0]), days=300)
print(fx1+fx2)
print(fx12+fx)

29.563038048499998
31.26310854225


In [23]:
def brute_optim_fn(total_resource=30, day0=30):
    days = day0+365
    max_val = 0
    max_params = {}
    params = []
    choice_pos = np.array([0.25, 0.5, 1])
    min_duration = 10
    max_duration = total_resource*np.array([4,2,1])

    for start in range(day0, day0+365):
        for choice in choice_pos:
            maxd = max_duration[np.where(choice_pos == choice)[0].item()]
            for duration in range(min_duration, maxd+1):
                params.append([np.array([start]), np.array([choice]), np.array([duration])])
                
    for start1 in range(day0, day0+365):
        for choice1 in choice_pos:
            maxd1 = max_duration[np.where(choice_pos == choice1)[0].item()]
            for duration1 in range(min_duration, maxd1+1):
                for start2 in range(start1+duration1, day0+365):
                    for choice2 in choice_pos:
                        maxd2 = max_duration[np.where(choice_pos == choice2)[0].item()] - int(duration1*(choice2/choice1))
                        for duration2 in range(min_duration, maxd2+1):
                            params.append([np.array([start1,start2]), np.array([choice1,choice2]), np.array([duration1,duration2])])
                            
    for i in range(len(params)):
        value = calculate_opt(intervention_day=params[i][0], intervention_duration=params[i][2], intervention_choice=params[i][1], days = 400)
        if(value>max_val):
            max_params['start_array'] = params[i][0]
            max_params['duration_array'] = params[i][2]
            max_params['choice_array'] = params[i][1]
            max_val = value        
            
    return(max_val, max_params)

In [20]:
brute_optim_fn()

(20.369375990250003,
 {'start_array': array([88]),
  'duration_array': array([120]),
  'choice_array': array([0.25])})

In [20]:
# remove non-resource-optimal-runs and do grid search
def smart_brute_optim_fn(total_resource=30, day0=30):
    days = day0+365
    max_val = 0
    max_params = {}
    params = []
    choice_pos = np.array([0.25, 0.5, 1])
    min_duration = 10
    max_duration = total_resource*np.array([4,2,1])

#     for start in range(day0, day0+365):
#         for choice in choice_pos:
#             maxd = max_duration[np.where(choice_pos == choice)[0].item()]
#             params.append([np.array([start]), np.array([choice]), np.array([maxd])])

#   Directly appending the best parameter for num_int=1.
    params.append([np.array([88]), np.array([0.25]), np.array([120])])
                
    for start1 in range(day0, day0+365, 10):
        for choice1 in choice_pos:
            maxd1 = max_duration[np.where(choice_pos == choice1)[0].item()]
            for duration1 in range(min_duration, maxd1+1, 5):
                for start2 in range(start1+duration1+10, day0+365, 10):
                    for choice2 in choice_pos:
                        maxd2 = max_duration[np.where(choice_pos == choice2)[0].item()] - int(duration1*(choice1/choice2))
                        duration2 = maxd2
                        params.append([np.array([start1,start2]), np.array([choice1,choice2]), np.array([duration1,duration2])])
                            
    print(len(params))
    for i in range(len(params)):
        value = calculate_opt(intervention_day=params[i][0], intervention_duration=params[i][2], intervention_choice=params[i][1], days = 400)
        if(value>max_val):
            max_params['start_array'] = params[i][0]
            max_params['duration_array'] = params[i][2]
            max_params['choice_array'] = params[i][1]
            max_val = value        
            
    return(max_val, max_params)

In [6]:
# run for num_int = 1
smart_brute_optim_fn()

1095


(20.369375990250003,
 {'start_array': array([88]),
  'duration_array': array([120]),
  'choice_array': array([0.25])})

In [21]:
# run for num_int = 2
smart_brute_optim_fn()

57517


(20.4273826375,
 {'start_array': array([ 30, 110]),
  'duration_array': array([ 10, 100]),
  'choice_array': array([0.5 , 0.25])})

In [58]:
def check(start_array, duration_array, choice_array, total_resource):
    for i in range(1,len(start_array)):
        if(start_array[i] <= start_array[i-1] + duration_array[i-1]):
            return(0)
    resource_spent = np.dot(duration_array, choice_array)
    if(resource_spent != total_resource):
        return(0)
    else:
        return(1)


def grid_search(total_resource=30, day0=30):
    max_val = 0
    max_params = {}
    params = []
    min_duration = 10
    max_duration = total_resource*np.array([4,2,1])
    
    num_int = 3
   
    if(num_int == 1):
        start_val = [start for start in range(day0, day0+365, 10)]
        choice_val = [0.25, 0.5, 1]
        duration_val = [duration for duration in range(10, 4*total_resource, 5)]
        start_tuple = list(itertools.product(start_val))
        choice_tuple = list(itertools.product(choice_val))
        duration_tuple = list(itertools.product(duration_val))
        inputs = list(itertools.product(start_tuple, duration_tuple, choice_tuple))
        for inp in inputs:
            start_array = np.array(inp[0])
            duration_array = np.array(inp[1])
            choice_array = np.array(inp[2])
            if(check(start_array, duration_array, choice_array, total_resource)):
                params.append([start_array, choice_array, duration_array])
        
    if(num_int == 2):
        start_val = [start for start in range(day0, day0+365, 10)]
        choice_val = [0.25, 0.5, 1]
        duration_val = [duration for duration in range(10, 4*total_resource, 5)]
        start_tuple = list(itertools.product(start_val, start_val))
        choice_tuple = list(itertools.product(choice_val, choice_val))
        duration_tuple = list(itertools.product(duration_val, duration_val))
        inputs = list(itertools.product(start_tuple, duration_tuple, choice_tuple))
        for inp in inputs:
            start_array = np.array(inp[0])
            duration_array = np.array(inp[1])
            choice_array = np.array(inp[2])
            if(check(start_array, duration_array, choice_array, total_resource)):
                params.append([start_array, choice_array, duration_array])
            
    if(num_int == 3):
        start_val = [start for start in range(day0, day0+365, 40)]
        choice_val = [0.25, 0.5, 1]
        duration_val = [duration for duration in range(10, 4*total_resource, 10)]
        start_tuple = list(itertools.product(start_val, start_val, start_val))
        choice_tuple = list(itertools.product(choice_val, choice_val, choice_val))
        duration_tuple = list(itertools.product(duration_val, duration_val, duration_val))
        inputs = list(itertools.product(start_tuple, duration_tuple, choice_tuple))
        for inp in inputs:
            start_array = np.array(inp[0])
            duration_array = np.array(inp[1])
            choice_array = np.array(inp[2])
            if(check(start_array, duration_array, choice_array, total_resource)):
                params.append([start_array, choice_array, duration_array])
                
    if(num_int == 4):
        start_val = [start for start in range(day0, day0+365, 40)]
        choice_val = [0.25, 0.5, 1]
        duration_val = [duration for duration in range(10, 4*total_resource, 20)]
        start_tuple = list(itertools.product(start_val, start_val, start_val, start_val))
        choice_tuple = list(itertools.product(choice_val, choice_val, choice_val, choice_val))
        duration_tuple = list(itertools.product(duration_val, duration_val, duration_val, duration_val))
        inputs = list(itertools.product(start_tuple, duration_tuple, choice_tuple))
        for inp in inputs:
            start_array = np.array(inp[0])
            duration_array = np.array(inp[1])
            choice_array = np.array(inp[2])
            if(check(start_array, duration_array, choice_array, total_resource)):
                params.append([start_array, choice_array, duration_array])
                
                
    if(num_int == 5):
        start_val = [start for start in range(day0, day0+365, 40)]
        choice_val = [0.25, 0.5, 1]
        duration_val = [duration for duration in range(10, 4*total_resource, 20)]
        start_tuple = list(itertools.product(start_val, start_val, start_val, start_val, start_val))
        choice_tuple = list(itertools.product(choice_val, choice_val, choice_val, choice_val, choice_val))
        duration_tuple = list(itertools.product(duration_val, duration_val, duration_val, duration_val, duration_val))
        inputs = list(itertools.product(start_tuple, duration_tuple, choice_tuple))
        for inp in inputs:
            start_array = np.array(inp[0])
            duration_array = np.array(inp[1])
            choice_array = np.array(inp[2])
            if(check(start_array, duration_array, choice_array, total_resource)):
                params.append([start_array, choice_array, duration_array])
        
                            
    print(len(params))
    for i in range(len(params)):
        value = calculate_opt(intervention_day=params[i][0], intervention_duration=params[i][2], intervention_choice=params[i][1], days = 400)
        if(value>max_val):
            max_params['start_array'] = params[i][0]
            max_params['duration_array'] = params[i][2]
            max_params['choice_array'] = params[i][1]
            max_val = value        
            
    return(max_val, max_params)    

In [53]:
#num_int = 1
grid_search()

74


(19.847822609749993,
 {'start_array': array([110]),
  'duration_array': array([60]),
  'choice_array': array([0.5])})

In [55]:
# num_int = 2
grid_search()

36667


(20.4273826375,
 {'start_array': array([ 30, 110]),
  'duration_array': array([ 10, 100]),
  'choice_array': array([0.5 , 0.25])})

In [59]:
#num_int = 3
grid_search()

23048


(20.31362790025,
 {'start_array': array([ 30,  70, 110]),
  'duration_array': array([ 10,  10, 100]),
  'choice_array': array([0.25, 0.25, 0.25])})